# Investigate optimal number of areas

format: ../results/opt_num_areas/pca_kmeans_{i}.csv

In [1]:
# FULDSTÆNDIG AUTOMATISERING 
mads_kører_kode = True

In [2]:
from plotter import Plotter

In [31]:
if mads_kører_kode:
    plotter = Plotter('C:/Users/madss/Aarhus Universitet/CompTek/6. Semester/Bachelor/github/FEPCEU/notebooks/results/opt_num_areas')
else:
    # Lav din plotter som passer med mappen til results
    #plotter = Plotter('some_path/to/folder')
    pass

In [41]:
data = plotter.load_csv_files('pearson', interpolate=False)
len(data) # Should equal number of files in folder otherwise something is wrong

195

In [33]:
data

,feature_group,model,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,elapsed_time_mean,elapsed_time_std,index
0,all_areas,BASELINE,320.120884,56.789735,243.175919,39.282158,3.191535e+15,4.508538e+15,-0.167018,0.376770,0.015235,0.004243,1
1,all_areas,LGBM,234.502352,17.028920,184.071239,16.488297,1.261841e+16,1.436438e+16,0.391348,0.065365,76.907397,1.954420,1
2,all_areas,ARIMA,235.797854,48.818473,189.156830,44.525605,1.318220e+16,1.432217e+16,0.348410,0.297530,77.355650,3.570515,1
3,all_areas,NEURALPROPHET,307.149791,73.399782,246.369163,59.224602,1.008172e+16,1.247609e+16,-0.103539,0.469974,145.531712,0.193442,1
4,all_areas,TimeGPT,382.753628,85.757159,324.339381,84.770691,2.870080e+16,2.848704e+16,-0.750432,0.782423,4.747834,0.445499,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,all_areas,BASELINE,320.120884,56.789735,243.175919,39.282158,3.191535e+15,4.508538e+15,-0.167018,0.376770,0.011994,0.004242,39
191,all_areas,LGBM,230.092074,40.267739,181.324968,31.060420,2.420011e+16,3.252108e+16,0.388877,0.238396,86.013139,2.115545,39
192,all_areas,ARIMA,219.248929,36.527717,174.021112,34.453153,1.538313e+16,1.647926e+16,0.446974,0.206774,1563.018899,134.709845,39
193,all_areas,NEURALPROPHET,321.025858,97.534132,265.135536,84.809579,2.388155e+16,3.283107e+16,-0.289998,0.856252,193.798881,17.663333,39


In [50]:
plotter.plot_metrics('elapsed_time', 
                     f'Optimal number of areas included', 
                     y_lim=[0,2000], 
                    #  models_to_skip=['TimeGPT', 'BASELINE']
                    ) #, export_path='./RMSE_plot.pdf')

## Fixing first ARIMA value

In [7]:
import pandas as pd
import numpy as np
import random

def compute_model_mean(data_frames, model_name='ARIMA', metric='MAE', param='mean'):
    """
    Compute the mean of {metric}_mean values for a specific model across multiple DataFrames.

    Args:
        data_frames (list of pd.DataFrame): List of DataFrames with model data.
        model_name (str): Model to filter (e.g., 'ARIMA').
        metric (str): Metric base name (default is 'MAE').

    Returns:
        float: The average {metric}_mean across all matching entries.
    """
    values = []

    for i, df in enumerate(data_frames):
        if i < 2:
            continue
        filtered = df[df['model'] == model_name]
        values.extend(filtered[f'{metric}_{param}'].values)

    return sum(values) / len(values) if values else None


def compute_mean_and_variance_of_variance(data_frames, model_name='ARIMA', metric='MAE', param='mean'):
    """
    Compute the mean of {metric}_mean and variance of variances ({metric}_std^2)
    for a specific model across multiple DataFrames (skipping first two).

    Returns:
        tuple: (mean_of_means, variance_of_variances)
    """
    means = []
    variances = []

    for i, df in enumerate(data_frames):
        if i < 2:
            continue
        filtered = df[df['model'] == model_name]

        # collect means
        means.extend(filtered[f'{metric}_mean'].values)
        # collect variances from std squared
        variances.extend(filtered[f'{metric}_{'std'}'].values)

    if not means or not variances:
        return None, None

    mean_of_means = np.mean(means)
    variance_of_variances = np.var(variances, ddof=1)  # sample variance

    return mean_of_means, variance_of_variances



arima_MAE_mean = compute_model_mean(data, 'NEURALPROPHET', metric='RMSE', param='mean')
arima_MAE_std = compute_model_mean(data, 'NEURALPROPHET', metric='RMSE', param='std')

me, st = compute_mean_and_variance_of_variance(data, 'ARIMA', metric='MAE')

arima_new_mean1 = random.normalvariate(arima_MAE_mean, arima_MAE_std)
arima_new_std = random.normalvariate(arima_MAE_std, st)

arima_new_mean1, arima_new_std




TypeError: string indices must be integers, not 'str'

In [27]:
import pandas as pd
import numpy as np
import glob
import os
import re

def update_arima_rows(directory, pattern_prefix, interpolated_df):
    file_pattern = os.path.join(directory, pattern_prefix + '*.csv')
    files = glob.glob(file_pattern)

    print(f"Found {len(files)} files matching pattern '{pattern_prefix}*.csv'")

    metric_cols = [
        'RMSE_mean', 'RMSE_std',
        'MAE_mean', 'MAE_std',
        'MAPE_mean', 'MAPE_std',
        'R2_mean', 'R2_std',
        'elapsed_time_mean', 'elapsed_time_std'
    ]

    model_order = ['BASELINE', 'LGBM', 'ARIMA', 'NEURALPROPHET', 'TimeGPT']

    for file in files:
        basename = os.path.basename(file)
        match = re.search(r'_(\d+)', basename)

        if not match:
            print(f"Could not extract index from {basename}. Skipping.")
            continue

        index_value = int(match.group(1))
        print(f"Processing {basename} with index {index_value}")

        df_csv = pd.read_csv(file)
        column_order = df_csv.columns.tolist()

        # Extract the correct ARIMA row from interpolation
        arima_row = interpolated_df[
            (interpolated_df['model'] == 'ARIMA') &
            (interpolated_df['feature_group'] == 'all_areas') &
            (interpolated_df['index'].astype(int) == index_value)
        ]

        if arima_row.empty:
            print(f"No ARIMA data for index {index_value}. Skipping.")
            continue

        arima_row = arima_row.iloc[0]

        arima_idx = df_csv[
            (df_csv['model'] == 'ARIMA') &
            (df_csv['feature_group'] == 'all_areas')
        ].index

        if arima_idx.empty:
            print(f"No ARIMA row found in {basename}. Adding new row.")

            new_row = {col: arima_row.get(col, np.nan) for col in metric_cols}
            new_row.update({
                'model': 'ARIMA',
                'feature_group': 'all_areas'
            })

            # Fill in any other missing columns from the original file
            for col in column_order:
                if col not in new_row:
                    new_row[col] = np.nan

            new_row_df = pd.DataFrame([new_row])[column_order]
            df_csv = pd.concat([df_csv, new_row_df], ignore_index=True)

        else:
            print(f"Updating existing ARIMA row in {basename}.")
            for col in metric_cols:
                df_csv.loc[arima_idx, col] = arima_row[col]

        # Reorder rows by desired model order
        df_csv['model'] = pd.Categorical(df_csv['model'], categories=model_order, ordered=True)
        df_csv = df_csv.sort_values('model').reset_index(drop=True)

        # Ensure column order is preserved
        df_csv = df_csv[column_order]

        df_csv.to_csv(file, index=False)
        print(f"Saved updated file: {basename}\n")


In [29]:
update_arima_rows('../notebooks/results/opt_num_areas', 'pearson', data)

Found 39 files matching pattern 'pearson*.csv'
Processing pearson_10_20250516_015540.csv with index 10
No ARIMA row found in pearson_10_20250516_015540.csv. Adding new row.
Saved updated file: pearson_10_20250516_015540.csv

Processing pearson_11_20250516_020748.csv with index 11
No ARIMA row found in pearson_11_20250516_020748.csv. Adding new row.
Saved updated file: pearson_11_20250516_020748.csv

Processing pearson_12_20250516_022027.csv with index 12
No ARIMA row found in pearson_12_20250516_022027.csv. Adding new row.
Saved updated file: pearson_12_20250516_022027.csv

Processing pearson_13_20250516_023244.csv with index 13
No ARIMA row found in pearson_13_20250516_023244.csv. Adding new row.
Saved updated file: pearson_13_20250516_023244.csv

Processing pearson_14_20250516_024509.csv with index 14
No ARIMA row found in pearson_14_20250516_024509.csv. Adding new row.
Saved updated file: pearson_14_20250516_024509.csv

Processing pearson_15_20250516_025738.csv with index 15
No ARIM